In [36]:
from torchvision.ops import box_iou
import os
import pandas as pd
import json
import torch
import numpy as np
from collections import defaultdict

In [37]:
BASE_DIR = "/mnt/Enterprise/safal/AI_assisted_microscopy_system/"

In [44]:
gt_annotation_file_fold_1 = os.path.join(
    BASE_DIR,
    "cysts_dataset_all/brightfield_sample/fold_5/brightfield_sample_coco_annos_val.json",
)

pred_annotation_file_fold_1 = os.path.join(
    BASE_DIR,
    "outputs/brightfield_sample/faster_rcnn_x101_32x8d_fpn_mstrain_3x_coco/fold_5/results.bbox.json"
)

In [45]:
def calculate_precision_recall_f1(pred_annotations_file, gt_annotations_file):
    # Precision x Recall is obtained individually by each class
    # Loop through each class and calculate the precision and recall

    # Precision = TP / (TP + FP)
    # Recall = TP / (TP + FN)
    gt_annotations = json.load(open(gt_annotations_file))
    pred_annotations = json.load(open(pred_annotations_file))

    gt_annotations_df = pd.DataFrame(gt_annotations["annotations"])
    pred_annotations_df = pd.DataFrame(pred_annotations)
    
    # change bbox width and height to x2, y2
    pred_annotations_df["bbox"] = pred_annotations_df["bbox"].apply(
        lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
    )
    gt_annotations_df["bbox"] = gt_annotations_df["bbox"].apply(
        lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
    )
    
    categories = sorted(gt_annotations_df.category_id.unique())

    # dataframe to store the precision, recall and f1 score for each class
    metrics_df = pd.DataFrame(
        columns=["category_id", "precision", "recall", "f1_score", "TP", "FP"]
    )

    for category in categories:
        # get the ground truth annotations for the current class
        gt_annotations_df_class = gt_annotations_df[
            gt_annotations_df.category_id == category
        ]
        # get the predicted annotations for the current class
        pred_annotations_df_class = pred_annotations_df[
            pred_annotations_df.category_id == category
        ]

        # sort the predicted annotations by score
        pred_annotations_df_class = pred_annotations_df_class.sort_values(
            by="score", ascending=False
        )

        # filter predictions with score > 0.3
        pred_annotations_df_class = pred_annotations_df_class[
            pred_annotations_df_class.score > 0.3
        ]

        true_positives_class = 0
        false_positives_class = 0

        # get image ids for the current class from both ground truth and predicted annotations
        image_ids = pred_annotations_df_class["image_id"].unique()
        images_len = len(image_ids)

        for image in image_ids:
            # get the ground truth annotations for the current image
            gt_annotations_df_image = gt_annotations_df_class[
                gt_annotations_df_class.image_id == image
            ]
            # get the predicted annotations for the current image
            pred_annotations_df_image = pred_annotations_df_class[
                pred_annotations_df_class.image_id == image
            ]

            # get the ground truth bounding boxes
            gt_bboxes = list(gt_annotations_df_image.bbox.values)
            gt_bboxes = torch.tensor(gt_bboxes)

            # get the predicted bounding boxes

            # only take the predicted bounding boxes which have a score > 0.3
            pred_bboxes = list(pred_annotations_df_image.bbox.values)
            pred_bboxes = torch.tensor(pred_bboxes)


            if len(gt_bboxes) == 0:
                false_positives_class += len(pred_bboxes)
                continue

            # get the intersection over union for each predicted bounding box
            ious = box_iou(gt_bboxes, pred_bboxes)

            # get the maximum iou for each ground truth bounding box
            max_ious, _ = torch.max(ious, dim=0)

            # get the indices of the predicted bounding boxes with iou > 0.5
            tp_indices = torch.where(max_ious >= 0.5)[0]
            # print(ious)

            # get the indices of the predicted bounding boxes with iou < 0.5
            fp_indices = torch.where(max_ious < 0.5)[0]

            # update the true positives and false positives
            true_positives_class += len(tp_indices)
            false_positives_class += len(fp_indices)

        # print actual number of ground truth annotations and predicted annotations for the current class
        print(
            "Actual:",
            gt_annotations_df_class.shape[0],
            "Predicted:",
            pred_annotations_df_class.shape[0],
        )

        # print true positives and false positives for the current class
        print(
            "True positives:",
            true_positives_class,
            "False positives:",
            false_positives_class,
        )
        # calculate the precision and recall
        precision = true_positives_class / (true_positives_class + false_positives_class)
        recall = true_positives_class / gt_annotations_df_class.shape[0]

        category_name = gt_annotations["categories"][category]["name"]
        print(f"Category: {category_name}, Precision: {precision}, Recall: {recall}")
        f1_score = 2 * (precision * recall) / (precision + recall)
        print(f"F1 Score: {f1_score}")

        metrics_df = metrics_df.append(
            {
                "category_id": category,
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score,
                "TP": true_positives_class,
                "FP": false_positives_class,
            },
            ignore_index=True,
        )

    return metrics_df

        


In [46]:
metrics_df_fold_1 = calculate_precision_recall_f1(pred_annotation_file_fold_1, gt_annotation_file_fold_1)

Actual: 153 Predicted: 197
True positives: 141 False positives: 56
Category: Crypto, Precision: 0.7157360406091371, Recall: 0.9215686274509803
F1 Score: 0.8057142857142857
Actual: 68 Predicted: 88
True positives: 62 False positives: 26
Category: Giardia, Precision: 0.7045454545454546, Recall: 0.9117647058823529
F1 Score: 0.794871794871795


/tmp/ipykernel_3964489/789971966.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(
/tmp/ipykernel_3964489/789971966.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(


In [47]:
metrics_df_fold_1

,category_id,precision,recall,f1_score,TP,FP
0,0.0,0.715736,0.921569,0.805714,141.0,56.0
1,1.0,0.704545,0.911765,0.794872,62.0,26.0


In [54]:
gt_annotation_file_fold_3 = os.path.join(
    BASE_DIR,
    "cysts_dataset_all/brightfield_sample/fold_4/brightfield_sample_coco_annos_val.json",
)

pred_annotation_file_fold_3 = os.path.join(
    BASE_DIR,
    "outputs/brightfield_sample/faster_rcnn_x101_32x8d_fpn_mstrain_3x_coco/fold_4/results.bbox.json"
)

In [55]:
metrics_df_fold_2 = calculate_precision_recall_f1(pred_annotation_file_fold_3, gt_annotation_file_fold_3)

Actual: 146 Predicted: 190
True positives: 128 False positives: 62
Category: Crypto, Precision: 0.6736842105263158, Recall: 0.8767123287671232
F1 Score: 0.7619047619047619
Actual: 66 Predicted: 87
True positives: 61 False positives: 26
Category: Giardia, Precision: 0.7011494252873564, Recall: 0.9242424242424242
F1 Score: 0.7973856209150327


/tmp/ipykernel_3964489/789971966.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(
/tmp/ipykernel_3964489/789971966.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(


In [56]:
metrics_df_fold_2

,category_id,precision,recall,f1_score,TP,FP
0,0.0,0.673684,0.876712,0.761905,128.0,62.0
1,1.0,0.701149,0.924242,0.797386,61.0,26.0
